In [1]:
import numpy as np
import csv,os,sys
from glob import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
data_dir = '/content/drive/MyDrive/Audio/Audio_Classification-MLSP/train/'
files = glob(data_dir+'*.npy')
#print(files)

# print(files)

labels = '/content/drive/MyDrive/Audio/Audio_Classification-MLSP/annotations.csv'
col_files = []
col_labels = []
with open(labels,'r') as fin:
    reader = csv.reader(fin)
    for row in reader:
        col_files.append(row[1])
        col_labels.append(row[2])
       
col_files=col_files[1:]
col_labels=col_labels[1:]
# print(col_files,col_labels)

In [3]:
categories = ['Bark','Crying_and_sobbing','Doorbell','Knock','Meow','Microwave_oven','Shatter','Siren','Vehicle_horn_and_car_horn_and_honking','Walk_and_footsteps']
print(len(categories))

10


In [4]:
def label_to_integer(arr):
    values = np.asarray(arr)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

categories_int = label_to_integer(categories)
print(categories_int)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [5]:
trainy = []
trainx = []
def get_one_hot(label_indx):
    indx = categories.index(label_indx)
    one_hot = categories_int[indx]
    return one_hot

for i in range(1000): #range(len(files)):
#     print(files[i])
    x = np.load(files[i])[0]
    #print(x.shape)
#ZERO_PADDING
    if x.shape[1]<=2584:
        len_pad = 2584-np.shape(x)[1]
        padf = np.zeros(shape=(128,len_pad))
        trainx.append(np.concatenate((x,padf),axis=1))
        indx = col_files.index(os.path.basename(files[i]))
        label_indx = col_labels[indx]
#    print(label_indx)
        trainy.append(get_one_hot(label_indx))

trainx = np.array(trainx)
trainy = np.array(trainy)

In [6]:
count=0
for i in range(1000):
  x = np.load(files[i])[0]
  if x.shape[1]>2432:
    count=count+1
    print(x.shape)
    print(os.path.basename(files[i]))
print(count)    

#TRIMMING
   ''' if x.shape[1]>2584:
      y=x[:,:2432] 
      trainx.append(y)  
#     print(x.shape)
#     print(os.path.basename(files[i]))'''

#INDEXES

IndentationError: ignored

In [6]:
print(np.shape(trainx),np.shape(trainy))

(1000, 128, 2584) (1000, 10)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainx, trainy, test_size=0.33, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(670, 128, 2584)
(670, 10)
(330, 128, 2584)
(330, 10)


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D,MaxPooling2D,Flatten,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers
from tensorflow import keras

In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D,MaxPooling2D,Flatten,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers
from tensorflow import keras

def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(128,2584,1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    #compile model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = define_model()
model.summary()
model.fit(trainx,trainy,validation_split=0.2,batch_size=32,verbose=1,epochs=100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 2430, 32)     320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 1215, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 1213, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 606, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 604, 32)       18464     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 302, 32)      0

KeyboardInterrupt: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential as sq
from tensorflow.keras.layers import Dense as ds, SimpleRNN as rn
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.0001)



model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 500)),
  #tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(16, activation='relu'),  
  tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(0.001),
#     loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
#     metrics=[tf.keras.metrics.CategoricalAccuracy()],
# )

model.fit(trainx,trainy,validation_split=0.2,batch_size=32,verbose=1,epochs=10)


# model=sq()

# model.add(ds(128*2584,input_dim =X_train.shape[0],activation='relu'))

# model.add(ds(209,activation='relu'))

# kernel_regularizer=keras.regularizers.l2(0.001)

# model.add(ds(102,activation='relu'))

# #kernel_regularizer=keras.regularizers.l2(0.001)

# model.add(ds(50,activation='relu'))

# tf.keras.layers.Dropout(0.2)

# kernel_regularizer=keras.regularizers.l2(0.001)

# model.add(ds(10,activation='softmax'))

# model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [9]:
def get_conv_model():
  model = Sequential()
  model.add(Conv2D(64, (3,3), activation='relu', strides=(1,1),padding='same', input_shape=(128, 2584, 1)))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.15))
  model.add(Conv2D(32, (3,3), activation='relu', strides=(1,1),padding='same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.5))
  model.add(Conv2D(32, (3,3), activation='relu', strides=(1,1),padding='same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(32, (3,3), activation='relu', strides=(1,1),padding='same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.35))
  model.add(Conv2D(32, (3,3), activation='relu', strides=(1,1),padding='same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Conv2D(64, (3,3), activation='relu', strides=(1,1),padding='same'))
  model.add(MaxPooling2D((4,4)))
  model.add(Dropout(0.5))
  model.add(Conv2D(64, (3,3), activation='relu', strides=(1,1),padding='same'))
  model.add(MaxPooling2D((1,1)))
  model.add(Dropout(0.45))
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(16, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  model.summary()
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [10]:
model=get_conv_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 2584, 64)     640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 1292, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 64, 1292, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 1292, 32)      18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 646, 32)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 32, 646, 32)       0

In [11]:

model.fit(X_train,y_train,validation_split=0.2,batch_size=32,verbose=1,epochs=100)

Epoch 1/100
17/17 [==============================] - 15s 595ms/step - loss: 2.2880 - accuracy: 0.1026 - val_loss: 2.2881 - val_accuracy: 0.1045
Epoch 2/100
17/17 [==============================] - 7s 426ms/step - loss: 2.2253 - accuracy: 0.1530 - val_loss: 2.2577 - val_accuracy: 0.1493
Epoch 3/100
17/17 [==============================] - 7s 426ms/step - loss: 2.1868 - accuracy: 0.1884 - val_loss: 2.2358 - val_accuracy: 0.1493
Epoch 4/100
17/17 [==============================] - 7s 427ms/step - loss: 2.1571 - accuracy: 0.2164 - val_loss: 2.2516 - val_accuracy: 0.1493
Epoch 5/100
17/17 [==============================] - 7s 428ms/step - loss: 2.1264 - accuracy: 0.2257 - val_loss: 2.2397 - val_accuracy: 0.2239
Epoch 6/100
17/17 [==============================] - 7s 430ms/step - loss: 2.0959 - accuracy: 0.2631 - val_loss: 2.2360 - val_accuracy: 0.2164
Epoch 7/100
17/17 [==============================] - 7s 431ms/step - loss: 2.0602 - accuracy: 0.2799 - val_loss: 2.1699 - val_accuracy: 0.231

In [13]:
model = get_conv_model()  # Get model (Sequential, Functional Model, or Model subclass)
model.save('/content/drive/MyDrive/Audio_Classification-MLSP')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 128, 2584, 64)     640       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 64, 1292, 64)     0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 64, 1292, 64)      0         
                                                                 
 conv2d_8 (Conv2D)           (None, 64, 1292, 32)      18464     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 32, 646, 32)      0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 32, 646, 32)      

In [12]:
# Evaluate the model with test data
#X_test=feature_extraction(X_test)
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

11/11 [==============================] - 2s 202ms/step - loss: 0.9579 - accuracy: 0.8000
Test loss: 0.9578734636306763
Test accuracy: 0.800000011920929


In [16]:
model_json = model.to_json()
with open("/content/drive/MyDrive/Audio_Classification-MLSP/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 

Saved model to disk


In [27]:
weights=model.get_weights()
len(weights)

20

In [28]:
for layer in model.layers:
    weights = layer.get_weights()

In [31]:
model.save_weights('/content/drive/MyDrive/Audio_Classification-MLSP/weightscnn80')

In [33]:
cnn=model.get_weights()